# Aaron's ISR challenge - are amp boundaries visible after ISR?
Implementing 20 pixel edge mask in PTC \
Craig Lage 25-Jan-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
def plotExp(exp, expId):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    ax1.set_title(f"{expId}")
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure, vmin, vmax

def plotArr(arr):
    # Borrowed from summit utils
    data = quickSmooth(arr, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    #ax1.set_title(f"{expId}")
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure, vmin, vmax

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return

def fitPlane(exp):
    x = np.arange(0,exp.image.array.shape[0])
    y = np.arange(0,exp.image.array.shape[1])
    X, Y = np.meshgrid(y, x)
    xRavel, yRavel = X.ravel(), Y.ravel()
    basis = []
    order = 1
    for i in range(order+1):
        for j in range(order - i +1):
            basis.append(xRavel**j * yRavel**i)
    A = np.vstack(basis).T
    b = exp.image.array.ravel()
    c, r, rank, s = np.linalg.lstsq(A, b, rcond=None)
    plane = c[0] + c[1] * X + c[2] * Y
    return plane
 
def correctGains(exp):
    plane = fitPlane(exp)
    xCut = 50
    yCut = 500
    corrs = {}
    ratioSum = 0
    for amp in exp.getDetector().getAmplifiers():
        ampName = amp.getName()
        bbox = amp.getBBox()
        xmin = bbox.beginX + xCut
        xmax = bbox.endX - xCut
        ymin = bbox.beginY + yCut
        ymax = bbox.endY - yCut
        imMedian = np.median(exp.image.array[ymin:ymax,xmin:xmax])
        planeMedian = np.median(plane[ymin:ymax,xmin:xmax])
        ratio = planeMedian / imMedian
        ratioSum += ratio
        corrs[ampName] = ratio
    factor = ratioSum / 16.0
    for amp in exp.getDetector().getAmplifiers():
        ampName = amp.getName()
        corrs[ampName] /= factor
        print(ampName, corrs[ampName])
    return corrs


In [ ]:
butler = Butler("/repo/main", collections=["LSSTCam/raw/all","u/cslage/calib/13144/calib.20220107"])
biasButler = Butler("/repo/main", collections=["u/cslage/bias_parallel_13144_B_17jan23"])
ptcButler = Butler("/repo/main", collections=["u/cslage/ptc_subregion_full_8C_12feb23"])

In [ ]:
expId = 3021120700188
detector = 55

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=True
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doBrighterFatter=False
isrConfig.doDark=True
isrConfig.doCrosstalk=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=True
isrConfig.doDefect=False
isrConfig.doDeferredCharge=True
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

## Run first to get the gain corrections

In [ ]:
# Zero the correction
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
corrs = {}
ratioSum = 0
for amp in exp.getDetector().getAmplifiers():
    ampName = amp.getName()
    corrs[ampName] = 1.00


In [ ]:
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
bias = biasButler.get('bias', detector=detector, exposure=expId, instrument="LSSTCam")
flat = butler.get('flat', detector=detector, exposure=expId, instrument="LSSTCam")
dark = butler.get('dark', detector=detector, exposure=expId, instrument="LSSTCam")
defects = butler.get('defects', detector=detector, exposure=expId, instrument="LSSTCam")
ptc = ptcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
linearizer = butler.get('linearizer', detector=detector, exposure=expId, instrument="LSSTCam")
deferredChargeCalib = butler.get('cpCtiCalib', detector=detector, exposure=expId, instrument="LSSTCam")


isrResult = isrTask.run(exp, bias=bias, flat=flat, dark=dark, defects=defects, \
                        ptc=ptc, linearizer=linearizer, deferredChargeCalib=deferredChargeCalib)

postIsrExp = isrResult.exposure
#repairCosmics(postIsrExp)
fig, vmin, vmax = plotExp(postIsrExp, expId)
plt.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/challenge_21jan23/Challenge_Subregions_{detector}_{expId}.png")

In [ ]:
plane = fitPlane(postIsrExp)
xmax = postIsrExp.image.array.shape[0]
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)
rawPlot = np.median(postIsrExp.image.array[2800:3200, 0:xmax], axis=0)
planePlot = np.median(plane[2800:3200, 0:xmax], axis=0)
plt.subplot(2,2,1)
plt.title(f"Median - Top - {expId}", fontsize=10)
plt.plot(rawPlot, label="PostISR")
plt.plot(planePlot, color='red', ls='--', lw=2, label="Plane Fit")
#for i in range(8):
#    x = 509 * i
#    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,xmax)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
rawPlot = np.median(postIsrExp.image.array[1800:2200, 0:xmax], axis=0)
planePlot = np.median(plane[1800:2200, 0:xmax], axis=0)
plt.subplot(2,2,3)
plt.title(f"Median - Bottom - {expId}", fontsize=10)
plt.plot(rawPlot, label="PostISR")
plt.plot(planePlot, color='red', ls='--', lw=2, label="Plane Fit")
#for i in range(8):
#    x = 509 * i
#    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,xmax)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

rawPlot = np.median(postIsrExp.image.array[0:4000, 1500:2500], axis=1)
planePlot = np.median(plane[0:4000, 1500:2500], axis=1)
plt.subplot(2,2,2)
plt.title(f"Median - Top/Bottom - {expId}", fontsize=10)
plt.plot(rawPlot, label="PostISR")
plt.plot(planePlot, color='red', ls='--', lw=2, label="Plane Fit")
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("Y coord (pixels)")
plt.ylabel("ADU")
plt.subplot(2,2,4)
plt.axis('off')
xtext = 0.2
ytext = 1.1
plt.text(xtext, ytext+.02, "Gain adjustments:")
for amp in exp.getDetector().getAmplifiers():
    ytext -= 0.08
    plt.text(xtext, ytext, f"{amp.getName()}: {(corrs[amp.getName()] - 1.0) * 100.0:.2f}%")

fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/challenge_21jan23/Challenge_Subregions_{detector}_{expId}_Scan.png")

## Now correct the gains and run again.

In [ ]:
corrs = correctGains(postIsrExp)

In [ ]:
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
bias = biasButler.get('bias', detector=detector, exposure=expId, instrument="LSSTCam")
flat = butler.get('flat', detector=detector, exposure=expId, instrument="LSSTCam")
dark = butler.get('dark', detector=detector, exposure=expId, instrument="LSSTCam")
defects = butler.get('defects', detector=detector, exposure=expId, instrument="LSSTCam")
ptc = ptcButler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
for amp in exp.getDetector().getAmplifiers():
    ptc.gain[amp.getName()] *= corrs[amp.getName()]
linearizer = butler.get('linearizer', detector=detector, exposure=expId, instrument="LSSTCam")
deferredChargeCalib = butler.get('cpCtiCalib', detector=detector, exposure=expId, instrument="LSSTCam")


isrResult_corr = isrTask.run(exp, bias=bias, flat=flat, dark=dark, defects=defects, \
                        ptc=ptc, linearizer=linearizer, deferredChargeCalib=deferredChargeCalib)


postIsrExp_corr = isrResult_corr.exposure
#repairCosmics(postIsrExp)
fig, vmin, vmax = plotExp(postIsrExp_corr, expId)
plt.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/challenge_21jan23/Challenge_Gain_Corrected_Subregions_{detector}_{expId}.png")#

In [ ]:
plane = fitPlane(postIsrExp_corr)
xmax = postIsrExp_corr.image.array.shape[0]
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)
rawPlot = np.median(postIsrExp_corr.image.array[2800:3200, 0:xmax], axis=0)
planePlot = np.median(plane[2800:3200, 0:xmax], axis=0)
plt.subplot(2,2,1)
plt.title(f"Median - Top - {expId}", fontsize=10)
plt.plot(rawPlot, label="PostISR")
plt.plot(planePlot, color='red', ls='--', lw=2, label="Plane Fit")
#for i in range(8):
#    x = 509 * i
#    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,xmax)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
rawPlot = np.median(postIsrExp_corr.image.array[1800:2200, 0:xmax], axis=0)
planePlot = np.median(plane[1800:2200, 0:xmax], axis=0)
plt.subplot(2,2,3)
plt.title(f"Median - Bottom - {expId}", fontsize=10)
plt.plot(rawPlot, label="PostISR")
plt.plot(planePlot, color='red', ls='--', lw=2, label="Plane Fit")
#for i in range(8):
#    x = 509 * i
#    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,xmax)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

rawPlot = np.median(postIsrExp_corr.image.array[0:4000, 1500:2500], axis=1)
planePlot = np.median(plane[0:4000, 1500:2500], axis=1)
plt.subplot(2,2,2)
plt.title(f"Median - Top/Bottom - {expId}", fontsize=10)
plt.plot(rawPlot, label="PostISR")
plt.plot(planePlot, color='red', ls='--', lw=2, label="Plane Fit")
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("Y coord (pixels)")
plt.ylabel("ADU")
plt.subplot(2,2,4)
plt.axis('off')
xtext = 0.2
ytext = 1.1
plt.text(xtext, ytext+.02, "Gain adjustments:")
for amp in exp.getDetector().getAmplifiers():
    ytext -= 0.08
    plt.text(xtext, ytext, f"{amp.getName()}: {(corrs[amp.getName()] - 1.0) * 100.0:.2f}%")

fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/challenge_21jan23/Challenge_Gain_Corrected_Edge_Subregions_{detector}_{expId}_Scan.png")